## Verification Policies

Verification policies are rules a verifier sets to specify what information they require from a holder to verify their identity or qualifications. They serve as a way for verifiers to communicate their verification requirements to holders and are used in the Present Proof protocol to construct proof requests.   

The PRISM Agent provides endpoints to create, update, fetch, lookup and delete verification policies. 
  
The policies attributes include:
- A unique ID
- A name
- The attributes to be verified
- The valid issuing DIDs (Decentralized Identifiers)
- The type of credentials that are required  

In [ ]:
import os
import time
import datetime
import base64
from pprint import pprint
from dotenv import load_dotenv

from prism_agent_open_api_specification_client import Client
from prism_agent_open_api_specification_client.models import VerificationPolicy, VerificationPolicyInput, VerificationPolicyPage
from prism_agent_open_api_specification_client.models import BadRequest, InternalServerError, NotFound
from prism_agent_open_api_specification_client.api.verification import create_verification_policy, delete_verification_policy_by_id, get_verification_policy_by_id
from prism_agent_open_api_specification_client.api.verification import update_verification_policy, lookup_verification_policies_by_query
from prism_agent_open_api_specification_client.types import Response


### Ultilitary functions

In [ ]:
def print_verification_policy(policy):
    print("id:", policy.id)
    print("name:", policy.name)
    print("created_at:", policy.created_at)
    print("updated_at:", policy.updated_at)
    print("attributes:", policy.attributes)
    print("issuer_di_ds:", policy.issuer_di_ds)
    print("credential_types:", policy.credential_types)
    print("additional_properties:", policy.additional_properties)
    
def print_verification_policy_page(verification_policy_page):
    for verification_policy in verification_policy_page.contents:
        print_verification_policy(verification_policy)
        print()

### Client instances

For this example we only need one client.

note: remember to update the file variables.env with the URLs and API keys provided to you.


In [ ]:
load_dotenv("../BetaProgram/variables.env")
verifierApiKey = os.getenv('VERIFIER_APIKEY')
verifierUrl = os.getenv('VERIFIER_URL')

verifier_client = Client(base_url=verifierUrl, headers={"apiKey": verifierApiKey})

print(f"Verifier URL:{verifierUrl}")

### Create Verification Policy

We will create two policies, one for driver's licenses and one for education requirements.

In [ ]:
data = {
    "name": f"Driver license verificaion {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    "attributes": [
        "name", "address", "type"
    ],
    "issuerDIDs": [
        "did:prism:1234"
    ],
    "credentialTypes": [
        "drivers license"
    ]
}
verification_policy_input = VerificationPolicyInput.from_dict(data)
verification_policy : [VerificationPolicy] = create_verification_policy.sync(client=verifier_client, json_body=verification_policy_input)

print(f"Drivers License: {verification_policy.id}\n")
print_verification_policy(verification_policy)


data_2 = {
    "name": f"Education requirement {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    "attributes": [
        "name", "address", "school", "title", "GPA"
    ],
    "issuerDIDs": [
        "did:prism:1234",
        "did:prism:4567",
        
    ],
    "credentialTypes": [
        "drivers license",
        "International drivers license"
    ]
}

verification_policy_input_2 = VerificationPolicyInput.from_dict(data_2)
verification_policy_2 : [VerificationPolicy] = create_verification_policy.sync(client=verifier_client, json_body=verification_policy_input_2)

print(f"\nEducation Requirement: {verification_policy_2.id}\n")
print_verification_policy(verification_policy_2)

### Update Verification Policy

Next, we will update the driver's License verification policy by adding an issuerDID.  
With the update endpoint the following fields can be updated: `attributes`, `issuerDIDs`, `name`, `credentialTypes`

In [ ]:
data = {
    "name": f"Driver license verificaion {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    "attributes": [
        "name", "address", "type"
    ],
    "issuerDIDs": [
        "did:prism:1234",
        "did:prism:4567",
        
    ],
    "credentialTypes": [
        "drivers license",
        "International drivers license"
    ]
}
verification_policy_input = VerificationPolicyInput.from_dict(data)
verification_policy : [VerificationPolicy] = update_verification_policy.sync(client=verifier_client, id=verification_policy.id, json_body=verification_policy_input)
print_verification_policy(verification_policy)

### Fetch

Get the verification policy by id

In [ ]:
verification_policy : [VerificationPolicy] = get_verification_policy_by_id.sync(client=verifier_client, id=verification_policy.id)
print_verification_policy(verification_policy)

### Lookup

The lookup endpoint can be used to get a list of verification policies. The available filters are `name`, `attributes`, `issuerDIDs`, and `credentialTypes` and control the pagination by `offset` and `limit` parameters.        
  

In this example, let's filter by attribute.  

In [ ]:
verification_policy_page : [verification_policy_page] = lookup_verification_policies_by_query.sync(client=verifier_client, attributes=["GPA"], limit=1)
print_verification_policy_page(verification_policy_page)

### Delete

Verification policies can be deleted by `id`

In [ ]:
verification_policy : [VerificationPolicy] = get_verification_policy_by_id.sync(client=verifier_client, id=verification_policy.id)
print("Get by id before deletion:\n")
print_verification_policy(verification_policy)

response = delete_verification_policy_by_id.sync(client=verifier_client, id=verification_policy.id)
response = get_verification_policy_by_id.sync(client=verifier_client, id=verification_policy.id)

print(f"\nGet by id after deletion: {response.msg}")